In [ ]:
import time
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import json
from activity_detector import ActivityDetector, LateralActivityHost, LateralActivityTarget, \
    LeadVehicle, ActivityDetectorParameters, LineData
from find_index_from_video import approx_index
from ngram import NGram
import seaborn as sns
from tqdm import tqdm
from typing import List, NamedTuple, Tuple
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [ ]:
# Load the data
i_file = 0
datafiles = glob(os.path.join("data", "1_hdf5", '*.hdf5'))
AD = ActivityDetector(datafiles[i_file])

In [ ]:
AD.set_lon_activities_host()

In [ ]:
AD.set_lat_activities_host()

The following codes checks the recall of the lane changed detection. It shows the activity at the index at which a lane change occurs. These indices are obtained after looking at the video. 

It shows that all left lane changes are detected. Two right lane changes are missed. 

In [ ]:
# Jeroen's code does not detect the lane change at 1963.60.
i_left_lane_change = [203.35, 217.29, 1738.83, 1813.3, 1900.71, 1963.60,
                      2000.3, 2171.98, 2310.65, 2318.23, 2546.01, 2650.02]
print([AD.get("host_lateral_activity", i) for i in i_left_lane_change])
# Jeroen's code does not detect the lane change at 2193.44 and 2236.72.
# These lane changes are also difficult to see by eye, so these indices
# are approximate.
i_right_lane_change = [251.46, 304.95, 1846.39, 1858.69, 1927.05, 2040.03,
                       2193.44, 2236.72, 2487.13, 2568.65, 2668.15, 2675.00]
print([AD.get("host_lateral_activity", i) for i in i_right_lane_change])

The following code shows the recall of the cut-in detection at the highway. There are six cut-ins identified from the videos. The lateral activities for these six target vehicles are determined. 
All cut-ins are detected. For some strange reason, the lane changes are detected 3 seconds before the timing that is obtained from the video. That might be a syncing error?

In [ ]:
# Cut-in targets:
# 3850: Motorbike from right (29:47, index=1802)
# 4171: Dark passenger car from right (quite far) (32:50, index=1985)
# 5028: Silver passenger car from right (39:18, index=2373)
# 5191: Black van from left (40:53, index=2468)
# 5645: Black passenger car from left (42:26, index=2562)
# 6008: Gray VW from left (44:26, index=2681)
for i_target, index in zip([3850, 4171, 5028, 5191, 5645, 6008],
                           [1802., 1985., 2373., 2468., 2562., 2681.]):
    AD.set_target_activities(i_target)
    print("Activity of target {:d} at index {:.2f}: {:s}".
          format(i_target, index-3, AD.targets[i_target].at[index-3, "lateral_activity"]))

Compute all events for the targets. This takes quite some time, around 13 minutes on the zBook laptop.

In [ ]:
# This takes round 13 minutes!
AD.set_target_activities()

## Create or load n-grams for target vehicles

In [ ]:
filename = os.path.join("data", "4_json", "{:s}_targets.json".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["longitudinal_activity", "lateral_activity", "longitudinal_state", 
              "lateral_state", "lead_vehicle"]
metadata = (("tstart", float), ("tend", float), ("target_id", int), ("target_tracker", int))
target_ngrams = NGram(fieldnames, metadata)

In [ ]:
if os.path.exists(filename):
    target_ngrams.from_json(filename)
else:
    for i_target in range(8):
        t_field_names = [AD.target_signal(i_target, fieldname) for fieldname in fieldnames]
        mapper = dict()
        for oldname, newname in zip(t_field_names, fieldnames):
            mapper[oldname] = newname
        dftargets = df[t_field_names].rename(columns=mapper)
        targetids = AD.get_t(i_target, "id")
        ids = np.unique(targetids)

        for targetid in ids[1:]:  # Skip the first one, which is always 0.
            dftarget = dftargets.loc[targetids == targetid]
            target_ngrams.ngram_from_data(dftarget, tstart=dftarget.index[0], tend=dftarget.index[-1],
                                          target_id=int(targetid), target_tracker=i_target)
    target_ngrams.sort_ngrams("tstart")
    target_ngrams.to_json(filename)

## Create or load n-gram for ego vehicle

In [ ]:
filename = os.path.join("data", "4_json", "{:s}_ego.json".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["host_longitudinal_activity", "host_lateral_activity",
              "is_highway"]
metadata = (("tstart", float), ("tend", float))
ego_ngram = NGram(fieldnames, metadata)

In [ ]:
if os.path.exists(filename):
    ego_ngram.from_json(filename)
else:
    ego_ngram.ngram_from_data(df, tstart=df.index[0], tend=df.index[-1])
    ego_ngram.to_json(filename)

## Extract cut-in scenario

For target vehicle, we need:
1. Lateral activity `li` or `ri`.
2. Lateral activity `fl` and lead vehicle `y`.

For the ego vehicle, we need it to go straight during step 2 of the target vehicle.

In [ ]:
def check_row(row, dict_tags: dict):
    """ Check if a row of a dataframe contains the provided tags. 
    
    Each item of the dictionary needs to contain a list. 
    
    :param row: The row that is obtained through pd.DataFrame.itertuples().
    :param dict_tags: The dictionary of tags.
    """
    for key, tags in dict_tags.items():
        if getattr(row, key) not in tags:
            return False
    return True

In [ ]:
target_tags = [dict(lateral_activity=[LateralActivityTarget.LEFT_CUT_IN.value, 
                                      LateralActivityTarget.RIGHT_CUT_IN.value]),
               dict(lateral_activity=[LateralActivityTarget.LANE_FOLLOWING.value], 
                    lead_vehicle=[LeadVehicle.LEAD.value])]
ego_tags = [dict(lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True])]

In [ ]:
i_start, i_step = 0, 0

ngram = target_ngrams.ngrams[365]
for row in ngram.data.itertuples():
    if i_step < len(target_tags) and check_row(row, target_tags[i_step]):
        print("Step {:d} at time {:.2f}".format(i_step+1, row.Index))

In [ ]:
# Tracker id is 1 and 4.
tstart = 2466
tend = 2466.5
for i in range(8):
    plt.plot(df.loc[tstart:tend, "Target_{:d}_dy".format(i)],
             df.loc[tstart:tend, "Target_{:d}_dx".format(i)], '.')
plt.xlim(-5, 5)
plt.ylim(0, 50)

In [ ]:
target_ngrams.ngrams[4000].meta["tstart"]

In [ ]:
df.loc[tstart:tend, ["Target_1_dx", "Target_4_dx", "Target_1_id", "Target_4_id"]]

In [ ]:
ego_ngram.ngrams[0].meta

In [ ]:
plt.plot(df.loc[2870:, "Host_vx"])